In [ ]:
s1 = 'café'
s2 = 'café'

In [ ]:
open('cafe.txt', 'w', encoding='utf8').write('café')

4

In [ ]:
open('cafe.txt').read()

'café'

In [ ]:
import sys, locale

expressions = """
    locale.getpreferredencoding()
    type(my_file)
    my_file.encoding
    sys.stdout.isatty()
    sys.stdout.encoding
    sys.stdin.isatty()
    sys.stdin.encoding
    sys.stderr.isatty()
    sys.stderr.encoding
    sys.getdefaultencoding()
    sys.getfilesystemencoding()
"""

my_file = open('dummy', 'w')

for expression in expressions.split():
    value = eval(expression)
    print(expression.rjust(30), '->', repr(value))


 locale.getpreferredencoding() -> 'UTF-8'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'UTF-8'
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> 'utf-8'
           sys.stderr.isatty() -> False
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'


In [ ]:
from unicodedata import normalize
len(normalize('NFC', s1)), len(normalize('NFC', s2))

(4, 4)

In [ ]:
len(normalize('NFD', s1)), len(normalize('NFD', s2))

(5, 5)

In [ ]:
normalize('NFC', s1), normalize('NFC', s2)

('café', 'café')

In [ ]:
normalize('NFD', s1), normalize('NFD', s2)

('café', 'café')

In [ ]:
normalize('NFC', s1) == normalize('NFC', s2)

True

In [ ]:
normalize('NFD', s1) == normalize('NFD', s2)

True

In [ ]:
from unicodedata import normalize, name

ohm = '\u2126'
name(ohm)

'OHM SIGN'

In [ ]:
ohm_c = normalize('NFC', ohm)
name(ohm_c)

'GREEK CAPITAL LETTER OMEGA'

In [ ]:
ohm, ohm_c, ohm == ohm_c

('Ω', 'Ω', False)

In [ ]:
normalize('NFC', ohm) == normalize('NFC', ohm_c)

True

In [ ]:
eszett = 'ß'
name(eszett)

'LATIN SMALL LETTER SHARP S'

In [ ]:
eszett_cf = eszett.casefold()
eszett_cf2 = eszett.lower()
eszett, eszett_cf, eszett_cf2

('ß', 'ss', 'ß')

In [ ]:
s1 = 'café'
s2 = 'cafe\u0301'  # U+0301 은 `COMBINING ACUTE ACCENT`
s1 == s2

False

In [ ]:
from unicodedata import normalize

def nfc_equal(str1, str2):
    return normalize('NFC', str1) == normalize('NFC', str2)

def fold_equal(str1, str2):
    return (normalize('NFC', str1).casefold() ==
            normalize('NFC', str2).casefold())

nfc_equal(s1, s2)

True

In [ ]:
nfc_equal('A', 'a')

False

In [ ]:
s3 = 'Straße'
s4 = 'strasse'

s3 == s4

False

In [ ]:
nfc_equal(s3, s4)

False

In [ ]:
fold_equal(s3, s4)

True

In [ ]:
fold_equal(s1, s2)

True

In [ ]:
fold_equal('A', 'a')

True

In [ ]:
import unicodedata
import string

def shave_marks(txt):
    """발음 구별 기호를 모두 제거한다"""
    norm_txt = unicodedata.normalize('NFD', txt)  # NFD 정규화
    shaved = ''.join(c for c in norm_txt
                     if not unicodedata.combining(c))  # 결합 문자 제거
    return unicodedata.normalize('NFC', shaved)  # NFC 정규화 후 반환

# 테스트 예제 1
order = '“Herr Voß: • ½ cup of Ötker™ caffè latte • bowl of açaí.”'
shave_marks(order)

'“Herr Voß: • ½ cup of Otker™ caffe latte • bowl of acai.”'

In [ ]:
# 테스트 예제 2
Greek = 'Ζέφυρος, Zéfiro'
shave_marks(Greek)

'Ζεφυρος, Zefiro'

In [ ]:
shave_marks('Straße')

'Straße'

In [ ]:
shave_marks('café')

'cafe'

In [ ]:
def shave_marks_latin(txt):

    norm_txt = unicodedata.normalize('NFD', txt)

    latin_base = False  # 라틴 문자 상태 초기화
    keepers = []  # 결과를 저장할 리스트

    for c in norm_txt:
        if unicodedata.combining(c) and latin_base:
            continue  # 라틴 문자의 결합 문자(악센트)를 무시

        keepers.append(c)  # 결합 문자가 아니면 리스트에 추가

        if not unicodedata.combining(c):
            latin_base = c in string.ascii_letters  # 새로운 기본 문자 여부 확인

    shaved = ''.join(keepers)  # 리스트를 문자열로 결합
    return unicodedata.normalize('NFC', shaved)  # NFC 정규화 후 반환


In [ ]:
single_map = str.maketrans(
    """‚ƒ„†ˆ‹‘’“”•–—˜›""",
    """'f"*^<''""---~>"""
)

multi_map = str.maketrans({
    '€': '<euro>',
    '…': '...',
    'Œ': 'OE',
    '™': '(TM)',
    'œ': 'oe',
    '‰': '<per mille>',
    '‡': '**',
})

multi_map.update(single_map)

def dewinize(txt):
    """Replace Win1252 symbols with ASCII chars or sequences"""
    return txt.translate(multi_map)

def asciize(txt):
    no_marks = shave_marks_latin(dewinize(txt))  # 결합 문자를 제거한 텍스트
    no_marks = no_marks.replace('ß', 'ss')  # ß → ss로 변환
    return unicodedata.normalize('NFKC', no_marks)  # NFKC 정규화 후 반환

In [ ]:
order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'
dewinize(order)

'"Herr Voß: - ½ cup of OEtker(TM) caffè latte - bowl of açaí."'

In [ ]:
asciize(order)

'"Herr Voss: - 1⁄2 cup of OEtker(TM) caffe latte - bowl of acai."'

In [ ]:
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted(fruits)

['acerola', 'atemoia', 'açaí', 'caju', 'cajá']

In [ ]:
!pip install pyuca

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 45.1 MB/s eta 0:00:00


In [ ]:
import pyuca

coll = pyuca.Collator()
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted_fruits = sorted(fruits, key=coll.sort_key)
sorted_fruits

['açaí', 'acerola', 'atemoia', 'cajá', 'caju']

In [ ]:
import unicodedata
import re

re_digit = re.compile(r'\d')
sample = '1\xbcb\u0969\u136b\u216b\u2466\u2480\u3285'

for char in sample:
    print('U+%04x' % ord(char),  # 유니코드 코드 포인트 출력
          char.center(6),         # 문자 가운데 정렬 (폭 6)
          're_dig' if re_digit.match(char) else '-',  # 정규식으로 숫자 확인
          'isdig' if char.isdigit() else '-',  # isdigit()으로 숫자 확인
          'isnum' if char.isnumeric() else '-',  # isnumeric()으로 숫자 확인
          format(unicodedata.numeric(char), '5.2f') if char.isnumeric() else '-',  # 숫자 값 출력
          unicodedata.name(char),  # 유니코드 문자 이름 출력
          sep='\t')  # 각 항목을 탭으로 구분

U+0031	  1   	re_dig	isdig	isnum	 1.00	DIGIT ONE
U+00bc	  ¼   	-	-	isnum	 0.25	VULGAR FRACTION ONE QUARTER
U+0062	  b   	-	-	-	-	LATIN SMALL LETTER B
U+0969	  ३   	re_dig	isdig	isnum	 3.00	DEVANAGARI DIGIT THREE
U+136b	  ፫   	-	isdig	isnum	 3.00	ETHIOPIC DIGIT THREE
U+216b	  Ⅻ   	-	-	isnum	12.00	ROMAN NUMERAL TWELVE
U+2466	  ⑦   	-	isdig	isnum	 7.00	CIRCLED DIGIT SEVEN
U+2480	  ⒀   	-	-	isnum	13.00	PARENTHESIZED NUMBER THIRTEEN
U+3285	  ㊅   	-	-	isnum	 6.00	CIRCLED IDEOGRAPH SIX


In [ ]:
import re

# 문자열에서 숫자와 단어를 찾기 위한 정규식 패턴
re_numbers_str = re.compile(r'\d+')
re_words_str = re.compile(r'\w+')

# 바이트 객체에서 숫자와 단어를 찾기 위한 정규식 패턴
re_numbers_bytes = re.compile(rb'\d+')
re_words_bytes = re.compile(rb'\w+')

# 문자열 및 바이트 객체 준비
text_str = (
    "Ramanujan saw \u0be7\u0bed\u0be8\u0bef"
    " as 1729 = 1³ + 12³ = 9³ + 10³."
)

text_bytes = text_str.encode('utf_8')  # 문자열을 UTF-8로 인코딩

# 출력
print('Text', repr(text_str), sep='\n ')
print('Numbers')
print(' str :', re_numbers_str.findall(text_str))  # 문자열에서 숫자 찾기
print(' bytes:', re_numbers_bytes.findall(text_bytes))  # 바이트 객체에서 숫자 찾기
print('Words')
print(' str :', re_words_str.findall(text_str))  # 문자열에서 단어 찾기
print(' bytes:', re_words_bytes.findall(text_bytes))  # 바이트 객체에서 단어 찾기

Text
 'Ramanujan saw ௧௭௨௯ as 1729 = 1³ + 12³ = 9³ + 10³.'
Numbers
 str : ['௧௭௨௯', '1729', '1', '12', '9', '10']
 bytes: [b'1729', b'1', b'12', b'9', b'10']
Words
 str : ['Ramanujan', 'saw', '௧௭௨௯', 'as', '1729', '1³', '12³', '9³', '10³']
 bytes: [b'Ramanujan', b'saw', b'as', b'1729', b'1', b'12', b'9', b'10']


In [ ]:
import os

os.listdir('.')

['.config',
 '.ipynb_checkpoints',
 'dummy',
 'fluent_python',
 'cafe.txt',
 'sample_data']

In [ ]:
os.listdir(b'.')

[b'.config',
 b'.ipynb_checkpoints',
 b'dummy',
 b'fluent_python',
 b'cafe.txt',
 b'sample_data']

In [ ]:
os.listdir('./fluent_python')

['.ipynb_checkpoints', 'digits-of-π.txt']

In [ ]:
os.listdir(b'./fluent_python')

[b'.ipynb_checkpoints', b'digits-of-\xcf\x80.txt']

In [ ]:
pi_name_bytes = os.listdir(b'.')[1]
pi_name_str = pi_name_bytes.decode('ascii', 'surrogateescape')
pi_name_str

'.ipynb_checkpoints'